In [1]:
!pip install torch torchvision transformers accelerate bitsandbytes fastapi uvicorn pyngrok deepspeed python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.4 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.16.5-py3-none-any.whl size=1580577 sha256=4090196c2e786c101905fe8a15f784a6be50411447566d487c97f88db9c4a47c
  Stored in directory: /root/.cache/pip/wheels/cb/fa/e7/98efc76db11fac734a4fae8c19dd08cc24257107e132e674f6
Successfully built deepspeed


In [2]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL.git
%cd DeepSeek-VL
!pip install -e .

Cloning into 'DeepSeek-VL'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 119 (delta 31), reused 15 (delta 15), pack-reused 66 (from 1)
Receiving objects: 100% (119/119), 12.47 MiB | 30.18 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/kaggle/working/DeepSeek-VL
Obtaining file:///kaggle/working/DeepSeek-VL
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for deepseek_vl (pyproject.toml) ... done
  Created wheel for deepseek_vl: filename=deepseek_vl-1.0.0-0.editable-py3-none-any.whl size=13187 sha256=7295faae18ba0cbd2095cb25fd7069fd99436e3e62281f3e0cbc35bdf1dc6d67
  Stored in directory: /tmp/pip-ephem-wheel-cache-3gscvswn/wheels/83/2f/30/ec3d2df9a70efd0182c8eb254c11d0a8b1daefee74cff331a3
Successfully bu

In [3]:
from fastapi import FastAPI, UploadFile, File,Form
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images
from PIL import Image
import torch
from io import BytesIO
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import threading

Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:524: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [5]:
# Configure bitsandbytes for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
device = "cuda"

In [6]:
model_path = "deepseek-ai/deepseek-vl-7b-chat"
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16  # Set the dtype here
)
device = vl_gpt.device

preprocessor_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


...

model.safetensors.index.json:   0%|          | 0.00/81.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.75G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
!fuser -k 8000/tcp

In [8]:
app = FastAPI()

@app.post("/analyze/")
async def analyze_image(image: UploadFile = File(...), prompt: str = Form(...)):
    """Processes a Base64 image and generates AI analysis."""
     # Read the image file
    contents = await image.read()
    image = Image.open(BytesIO(contents)).convert("RGB")

    # Save the image temporarily (optional)
    image_path = "./temp_image.png"
    image.save(image_path)

    # Prepare the conversation input
    conversation = [
        {
            "role": "User",
            "content": f"<image_placeholder>{prompt}",
            "images": [image_path]
        },
        {
            "role": "Assistant",
            "content": ""
        }
    ]

    # Load images and prepare inputs
    pil_images = load_pil_images(conversation)
    prepare_inputs = vl_chat_processor(
        conversations=conversation,
        images=pil_images,
        force_batchify=True
    ).to(vl_gpt.device)

    # Clear cache to free up memory
    torch.cuda.empty_cache()

    # Run image encoder to get the image embeddings
    inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

    # Run the model to get the response
    outputs = vl_gpt.language_model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=prepare_inputs.attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=1000,
        do_sample=False,
        use_cache=True
    )

    # Decode the response
    answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return {"analysis": answer}

In [9]:
!ngrok config add-authtoken 2v1YXcoc3UN1qNZDijJ56gZYoMe_6CajMtmWBSUdNf8Avwec7

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [10]:
# Start FastAPI server in a separate thread
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Expose the FastAPI port via ngrok
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

INFO:     Started server process [31]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://9ab7-34-136-78-202.ngrok-free.app
